## **필요 라이브러리 설치**

In [ ]:
# install dependencies: (use cu111 because colab has CUDA 11.1)
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html

# Install mmdetection
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

# Install roboflow
!pip install roboflow

!pip install -e .

In [ ]:
import os.path as osp
import mmcv

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

from roboflow import Roboflow

## **구글 드라이브 마운트**

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

## **데이터셋 다운로드**

In [ ]:
rf = Roboflow(api_key="IW5oromkiHkIl17GXOjc")
project = rf.workspace("sketch").project("sketch-detection")
dataset = project.version(2).download("coco")

## **공통 정보(변수, 함수..)**

In [ ]:
# 데이터셋 위치
data_root = '/content/mmdetection/sketch-detection-2/'

# 클래스 종류 
classes = ('button', 'checkbox_checked', 'checkbox_unchecked',
          'chip', 'data_table', 'dropdown_menu',
          'floating_action_button', 'image', 'radiobutton_checked',
          'radiobutton_unchecked', 'slider', 'switch_disabled',
          'switch_enabled', 'text_field', 'text_area')

# 클래스 수
num_classes = 15

# 어노테이션 파일명 함수 
ann_file = lambda x: f'{data_root}{x}/_annotations.coco.json'

# 이미지 프리픽스 함수 
img_prefix = lambda x: f'{data_root}{x}/'

# 데이터 설정 함수
def get_data(data, dataType):
  data.ann_file = ann_file(dataType)
  data.img_prefix = img_prefix(dataType)
  data.classes = classes
  return data

# 학습율
lr = 0.0025

# 학습 횟수
epochs = 30

# 로그 출력 간격
log_interval = 100

# checkpoint 저장 간격
checkpoint_interval = 1

# 평가 간격
eval_interval = 1

# 공통 config 설정 함수
def set_config(model, base):
  # base config 정보
  cfg = Config.fromfile(base)

  # 데이터 위치 설정
  cfg.data_root = data_root

  # Batch size of a single GPU
  cfg.data.samples_per_gpu = 2
  # Worker to pre-fetch data for each single GPU
  cfg.data.workers_per_gpu = 2

  # 학습율 설정 
  cfg.optimizer.lr = lr

  # 학습 횟수 설정
  cfg.runner.max_epochs = epochs

  # 워킹 디렉터리 설정
  cfg.work_dir = f'working/{model}'

  # gpu 설정
  cfg.gpu_ids = [0]
  cfg.device = 'cuda'

  # seed 설정
  cfg.seed = 0

  # 로그 간격 설정
  cfg.log_config.interval = log_interval

  # checkpoint 저장 간격 설정
  cfg.checkpoint_config.interval = checkpoint_interval

  # 평가 간격 설정
  cfg.evaluation.interval = eval_interval

  return cfg 
  
# 학습 함수
def train(cfg):
  # Build dataset
  datasets = [build_dataset(cfg.data.train)]

  # Build the detector
  model = build_detector(cfg.model)
  # Add an attribute for visualization convenience
  model.CLASSES = datasets[0].CLASSES

  # Create work_dir
  mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))

  # Train
  train_detector(model, datasets, cfg, distributed=False, validate=True)

## **Faster R-CNN**

In [ ]:
# 공통 config 설정
cfg = set_config('faster_rcnn', './configs/faster_rcnn/faster_rcnn_r50_caffe_fpn_mstrain_1x_coco.py')

# 학습 데이터 설정 
cfg.data.train = get_data(cfg.data.train, 'train')

# 검증 데이터 설정
cfg.data.val = get_data(cfg.data.val, 'valid')

# 테스트 데이터 설정
cfg.data.test = get_data(cfg.data.test, 'test')

# 클래스 수 설정
cfg.model.roi_head.bbox_head.num_classes = num_classes

# loss 함수 변경
cfg.model.roi_head.bbox_head.loss_cls=dict(
                                type='FocalLoss',
                                use_sigmoid=True,
                                gamma=1.5,
                                alpha=0.5,
                                loss_weight=1.0)

# config 내용 파일에 저장 
with open('config.py', 'w') as f:
  f.write(cfg.pretty_text)

# config 파일 백업 
mmcv.mkdir_or_exist('/content/drive/MyDrive/model/faster_rcnn')
mmcv.mkdir_or_exist('./working/faster_rcnn')
!cp config.py /content/drive/MyDrive/model/faster_rcnn/
!mv config.py working/faster_rcnn/

In [ ]:
# 학습 
train(cfg)

# checkpoint 백업 
!cp -r /content/mmdetection/working/faster_rcnn /content/drive/MyDrive/model

In [ ]:
# evaluate - bbox
!python tools/test.py \
        working/faster_rcnn/config.py \
        working/faster_rcnn/epoch_30.pth \
        --out working/faster_rcnn/result.pkl \
        --eval bbox \

!cp working/faster_rcnn/result.pkl ../drive/MyDrive/model/faster_rcnn/

In [ ]:
# evaluate - confusion matrix
!python tools/analysis_tools/confusion_matrix.py \
        working/faster_rcnn/config.py \
        working/faster_rcnn/result.pkl \
        working/faster_rcnn/

!cp working/faster_rcnn/confusion_matrix.png ../drive/MyDrive/model/faster_rcnn/

## **TOOD**

In [ ]:
# 공통 config 설정
cfg = set_config('tood', './configs/tood/tood_r101_fpn_dconv_c3-c5_mstrain_2x_coco.py')

# 학습 데이터 설정 
cfg.data.train = get_data(cfg.data.train, 'train')

# 검증 데이터 설정
cfg.data.val = get_data(cfg.data.val, 'valid')

# 테스트 데이터 설정
cfg.data.test = get_data(cfg.data.test, 'test')

# 클래스 수 설정
cfg.model.bbox_head.num_classes = num_classes

# error handling - The testing results of the whole dataset is empty.
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

# warmup 설정 
cfg.lr_config.warmup = None

# config 내용 파일에 저장 
with open('config.py', 'w') as f:
  f.write(cfg.pretty_text)

# config 파일 백업 
mmcv.mkdir_or_exist('/content/drive/MyDrive/model/tood')
mmcv.mkdir_or_exist('./working/tood')
!cp config.py /content/drive/MyDrive/model/tood/
!mv config.py working/tood/

In [ ]:
# 학습 
train(cfg)

# checkpoint 백업 
!cp -r /content/mmdetection/working/tood /content/drive/MyDrive/model

In [ ]:
# evaluate - bbox
!python tools/test.py \
        working/tood/config.py \
        working/tood/epoch_30.pth \
        --out working/tood/result.pkl \
        --eval bbox \

!cp working/tood/result.pkl ../drive/MyDrive/model/tood/

In [ ]:
# evaluate - confusion matrix
!python tools/analysis_tools/confusion_matrix.py \
        working/tood/config.py \
        working/tood/result.pkl \
        working/tood/

!cp working/tood/confusion_matrix.png ../drive/MyDrive/model/tood/

## **YOLOF**

In [ ]:
# 공통 config 설정
cfg = set_config('yolof', './configs/yolof/yolof_r50_c5_8x8_1x_coco.py')

# 학습 데이터 설정 
cfg.data.train = get_data(cfg.data.train, 'train')

# 검증 데이터 설정
cfg.data.val = get_data(cfg.data.val, 'valid')

# 테스트 데이터 설정
cfg.data.test = get_data(cfg.data.test, 'test')

# 클래스 수 설정
cfg.model.bbox_head.num_classes = 15

# error handling - The testing results of the whole dataset is empty.
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

# config 내용 파일에 저장 
with open('config.py', 'w') as f:
  f.write(cfg.pretty_text)

# config 파일 백업 
mmcv.mkdir_or_exist('/content/drive/MyDrive/model/yolof')
mmcv.mkdir_or_exist('./working/yolof')
!cp config.py /content/drive/MyDrive/model/yolof/
!mv config.py working/yolof/

In [ ]:
# 학습 
train(cfg)

# checkpoint 백업 
!cp -r /content/mmdetection/working/yolof /content/drive/MyDrive/model

In [ ]:
# evaluate - bbox
!python tools/test.py \
        working/yolof/config.py \
        working/yolof/epoch_30.pth \
        --out working/yolof/result.pkl \
        --eval bbox \

!cp working/yolof/result.pkl ../drive/MyDrive/model/yolof/

In [ ]:
# evaluate - confusion matrix
!python tools/analysis_tools/confusion_matrix.py \
        working/yolof/config.py \
        working/yolof/result.pkl \
        working/yolof/

!cp working/yolof/confusion_matrix.png ../drive/MyDrive/model/yolof/

## **YOLOX**

In [ ]:
# 공통 config 설정
cfg = set_config('yolox', './configs/yolox/yolox_l_8x8_300e_coco.py')

# 학습 데이터 설정 
cfg.data.train.dataset = get_data(cfg.data.train.dataset, 'train')
cfg.train_dataset.dataset = get_data(cfg.train_dataset.dataset, 'train')

# 검증 데이터 설정
cfg.data.val = get_data(cfg.data.val, 'valid')

# 테스트 데이터 설정
cfg.data.test = get_data(cfg.data.test, 'test')

# 클래스 수 설정
cfg.model.bbox_head.num_classes = num_classes

# error handling - The testing results of the whole dataset is empty.
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

# config 내용 파일에 저장 
with open('config.py', 'w') as f:
  f.write(cfg.pretty_text)

# config 파일 백업 
mmcv.mkdir_or_exist('/content/drive/MyDrive/model/yolox')
mmcv.mkdir_or_exist('./working/yolox')
!cp config.py /content/drive/MyDrive/model/yolox/
!mv config.py working/yolox/

In [ ]:
# 학습 
train(cfg)

# checkpoint 백업 
!cp -r /content/mmdetection/working/yolox /content/drive/MyDrive/model

In [ ]:
# evaluate - bbox
!python tools/test.py \
        working/yolox/config.py \
        working/yolox/epoch_30.pth \
        --out working/yolox/result.pkl \
        --eval bbox \

!cp working/yolox/result.pkl ../drive/MyDrive/model/yolox/

In [ ]:
# evaluate - confusion matrix
!python tools/analysis_tools/confusion_matrix.py \
        working/yolox/config.py \
        working/yolox/result.pkl \
        working/yolox/

!cp working/yolox/confusion_matrix.png ../drive/MyDrive/model/yolox/

## **RetinaNet**

In [ ]:
# 공통 config 설정
cfg = set_config('retinanet', './configs/retinanet/retinanet_x101_64x4d_fpn_1x_coco.py')

# 학습 데이터 설정 
cfg.data.train = get_data(cfg.data.train, 'train')

# 검증 데이터 설정
cfg.data.val = get_data(cfg.data.val, 'valid')

# 테스트 데이터 설정
cfg.data.test = get_data(cfg.data.test, 'test')

# 클래스 수 설정
cfg.model.bbox_head.num_classes = 15

# error handling - The testing results of the whole dataset is empty.
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

# config 내용 파일에 저장 
with open('config.py', 'w') as f:
  f.write(cfg.pretty_text)

# config 파일 백업 
mmcv.mkdir_or_exist('/content/drive/MyDrive/model/retinanet')
mmcv.mkdir_or_exist('./working/retinanet')
!cp config.py /content/drive/MyDrive/model/retinanet/
!mv config.py working/retinanet/

In [ ]:
# 학습 
train(cfg)

# checkpoint 백업 
!cp -r /content/mmdetection/working/retinanet /content/drive/MyDrive/model/retinanet

In [ ]:
# evaluate - bbox
!python tools/test.py \
        working/retinanet/config.py \
        working/retinanet/epoch_30.pth \
        --out working/retinanet/result.pkl \
        --eval bbox \

!cp working/retinanet/result.pkl ../drive/MyDrive/model/retinanet/

In [ ]:
# evaluate - confusion matrix
!python tools/analysis_tools/confusion_matrix.py \
        working/retinanet/config.py \
        working/retinanet/result.pkl \
        working/retinanet/

!cp working/retinanet/confusion_matrix.png ../drive/MyDrive/model/retinanet/